# Model experiments - sample set

In the previous notebooks we have separated a small subset of our data, called "sample", on which we can now experiment with simple models to assess the effectiveness of our preprocessing & data augmentation techniques.

We do it this way to avoid spending too much time on training on the entire set, the assumption is that the methods which are effective on the sample will work well on a larger scale too. 

We will start by testing a couple of simple models on untouched sample data (as numpy arrays) and then proceed towards data augmentation and finally spectrograms.

In [1]:
# first make sure we're in the parent dictory of our data/sample folders.
!pwd

/home/paperspace/tensorflow_speech_recognition


## Import
We'll need a couple of additional libraries so let's import them.

In [2]:
# filter out warnings
import warnings
warnings.filterwarnings('ignore') 

In [21]:
import glob
import librosa
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow

# utils
from importlib import reload
import utils; reload(utils)

# keras as tensorflow backend
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, BatchNormalization, Dropout, Convolution1D, MaxPooling1D, Flatten
from tensorflow.python.keras.optimizers import Adam

# F1 and accuracy score metric
from sklearn.metrics import f1_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier

## Prepare data
The easiest way to work with data is by turning it into a list of numbers, in our case a numpy array. We can use one of the functions from utils to load the raw data or use the librosa.load() function. The difference lies in the fact that the former returns int16s whereas librosa returns float32s and uses its default sampling rate of 22050Hz, unless we explicitly tell it to use the file's original sampling rate of 16000Hz.

We should also consider normalizing our data (so that it all falls within the same scale) and using the preprocessing methods explored in the previous notebook (MFCCs, Mel spectrogram, fast fourier transform and tempogram). 

In [4]:
path_to_sample = "data/sample"

We'll have to go through each of the folders in our sample/train, cv and test sets, one-hot encode their label and load the 16K long array of raw data. The y data will be of shape (m, 12), where m is the number of examples, and the X data will be of shape (m, 16000) - at least for the raw .wav input.

Let's calculate **m** first. We will do that by using a function that create a list of all the .wav files within a directory.

### Create a list of paths
We will use the glob module that we learned about in the very first notebook and a function from util.py which can, given a directory, return a list of paths to .wav files within it. We will repeat the process for all 3 sets within sample, and every category subdirectory within those too.

In [5]:
# for example we can grab all .wav files from sample/train/stop
path_to_sample_train_stop = os.path.join(path_to_sample, "train", "stop")
utils.grab_wavs(path_to_sample_train_stop)[:5]

['data/sample/train/stop/01b4757a_nohash_0.wav',
 'data/sample/train/stop/3ac2e76f_nohash_0.wav',
 'data/sample/train/stop/3e31dffe_nohash_3.wav',
 'data/sample/train/stop/37bd115d_nohash_1.wav',
 'data/sample/train/stop/6c2dd2d5_nohash_0.wav']

In [6]:
# we'll need a list of all category folder names
categories_to_predict = ["yes", "no", "up", "down", "left", "right", "on", "off", "stop", "go", "silence", "unknown"]

In [7]:
# first grab the training set
path_to_train = os.path.join(path_to_sample, "train")
sample_train_wavs = []

for category in categories_to_predict:
    path_to_category = os.path.join(path_to_train, category)
    category_files = utils.grab_wavs(path_to_category)
    
    # we use extend instead of append to add all elements from the iterable
    sample_train_wavs.extend(category_files)
    
sample_train_wavs

['data/sample/train/yes/0f3f64d5_nohash_0.wav',
 'data/sample/train/yes/8a28231e_nohash_3.wav',
 'data/sample/train/yes/d3f22f0e_nohash_0.wav',
 'data/sample/train/yes/2d3c8dcb_nohash_1.wav',
 'data/sample/train/yes/d486fb84_nohash_0.wav',
 'data/sample/train/yes/61d3e51e_nohash_0.wav',
 'data/sample/train/yes/8f811bbc_nohash_0.wav',
 'data/sample/train/yes/b43de700_nohash_0.wav',
 'data/sample/train/yes/66a412a7_nohash_0.wav',
 'data/sample/train/yes/92e17cc4_nohash_0.wav',
 'data/sample/train/yes/6c9223bd_nohash_0.wav',
 'data/sample/train/yes/d5356b9a_nohash_0.wav',
 'data/sample/train/yes/e7d0eb3f_nohash_1.wav',
 'data/sample/train/yes/712e4d58_nohash_2.wav',
 'data/sample/train/yes/a2fefcb4_nohash_0.wav',
 'data/sample/train/yes/324210dd_nohash_1.wav',
 'data/sample/train/yes/742d6431_nohash_0.wav',
 'data/sample/train/yes/70a00e98_nohash_1.wav',
 'data/sample/train/yes/28e47b1a_nohash_4.wav',
 'data/sample/train/yes/7014b07e_nohash_1.wav',
 'data/sample/train/no/f0ebef1b_nohash_0

In [8]:
# repeat for cv
path_to_cv = os.path.join(path_to_sample, "cv")
sample_cv_wavs = []

for category in categories_to_predict:
    path_to_category = os.path.join(path_to_cv, category)
    category_files = utils.grab_wavs(path_to_category)
    sample_cv_wavs.extend(category_files)

# repeat for test
path_to_test = os.path.join(path_to_sample, "test")
sample_test_wavs = []

for category in categories_to_predict:
    path_to_category = os.path.join(path_to_test, category)
    category_files = utils.grab_wavs(path_to_category)
    sample_test_wavs.extend(category_files)

### One-hot encode the y

Now that we have the 3 lists of files from each set (train, cv and test) we can construct our train_y, cv_y and test_y numpy arrays. These will be matrices of size (m, 12), one-hot encoded. E.g. if a row belongs to the category "up" it will take the form of an array of zeros, where the entry at index 2 (the third from the left) will become a 1.

We will use a function from the utils that takes a path to a .wav, the index at which the category name starts within it (we want to control this because we will eventually use this for the main set, not just the sample) and a list of categories to predict. For our current example, the category name in the paths belonging to "train" starts at the 18th index (separators count as one char).

In [9]:
# let's grab a single path (this one is an "left")
a_wav = sample_train_wavs[80]
a_wav

'data/sample/train/left/4ec7d027_nohash_0.wav'

In [10]:
# let's see if the 1 is correctly placed
utils.one_hot_encode_path(a_wav, 18, categories_to_predict)

array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])

The path belonged to the fifth category ("left") and the one-hot encoding correctly placed the 1 at index 4 (zero-indexed).

We want to repeat this for all examples in each of the 3 subsets, adding each new one-hot encoded numpy array as a new row of the y matrix, in order.

In [11]:
# figure out the dimensions of train_y
rows = len(sample_train_wavs)
columns = len(categories_to_predict)
dimensions = (rows, columns)
dimensions

(240, 12)

In [12]:
# create train_y as empty array
train_y = np.array([])

# append each row to train_y
for path_to_wav in sample_train_wavs:
    row = utils.one_hot_encode_path(path_to_wav, 18, categories_to_predict)
    
    # append the new row
    train_y = np.append(train_y, row)
    
# we currently have a flattened vector
print("Current shape: {}".format(*train_y.shape))

# let's reshape it
train_y = np.reshape(train_y, dimensions)
print("New shape: {}".format(train_y.shape))

Current shape: 2880
New shape: (240, 12)


In [13]:
# show the train_y matrix to confirm
train_y

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

We can see that the first 3 entries have the 1 at 0th index, which means they belong to category "up" and the last three have the 1 at the last index, which is also correct given the fact that our list of paths was also ordered.

We should bear in mind that by default the np.array contains float64s and our functions for loading a .wav return int16s.

Since this is a highly-repetitive task we'll want to use the utils function for obtaining the y.

Repeat for **CV set**.

In [14]:
# figure out the dimensions
rows = len(sample_cv_wavs)
columns = len(categories_to_predict)
dimensions = (rows, columns)
print("Target dimensions: {}".format(dimensions))

# get the y
cv_y = utils.get_y(sample_cv_wavs, 15, categories_to_predict)
print("Received shape: {}".format(cv_y.shape))

Target dimensions: (60, 12)
Received shape: (60, 12)


Repeat for **Test set**.

In [15]:
# figure out the dimensions
rows = len(sample_test_wavs)
columns = len(categories_to_predict)
dimensions = (rows, columns)
print("Target dimensions: {}".format(dimensions))

# get the y
test_y = utils.get_y(sample_test_wavs, 17, categories_to_predict)
print("Received shape: {}".format(test_y.shape))

Target dimensions: (60, 12)
Received shape: (60, 12)


In [16]:
test_y[0]

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

### Get the X
We have the y - the one-hot encoded vectors representing the category for each training, cv and test example in the sample set. We need the feature vectors, conventionally referred to as X. We will use both the simplest way of extracting the .wav data and the preprocessing techniques - MFCCs, Mel spectrogram, FFT and tempogram.

Let's start by defining a simple helper function for just the raw .wav data. Since our samples are of slightly differing lengths but each row of our X always has to have the same length, we will **add padding by default.**

In [17]:
# get the desired number of columns (n)
n = len(utils.get_wav_info(path_to_wav)[1])
n

16000

#### Raw .wav data

In [18]:
# define a simple helper function
def get_X_with_padding(list_of_paths, columns=16000):
    
    # get shape data
    rows = len(list_of_paths)
    dimensions = (rows, columns)
    
    # create placeholder
    X = np.array([])
    
    # go through every file path in the list
    for path_to_wav in list_of_paths:

        # get raw array of signed ints
        row = utils.get_wav_info(path_to_wav)[1]
        
        # some of our sample have less (or slightly more) than 16000 values, so let's adjust them
        # trim to fixed length
        row = row[:columns]
        
        # pad with zeros, calculating amount of padding needed
        padding = columns - len(row)
        row = np.pad(row, (0, padding), mode='constant', constant_values=0)

        # append the new row
        X = np.append(X, row)
    
    # reshape (unroll)
    X = np.reshape(X, dimensions)
    
    return X

In [19]:
# get the X for each set
train_X = utils.get_X(sample_train_wavs, n)
cv_X = utils.get_X(sample_cv_wavs, n)
test_X = utils.get_X(sample_test_wavs, n)

print("Train: ", train_X.shape)
print("CV: ", cv_X.shape)
print("Test: ",test_X.shape)

Train:  (240, 16000)
CV:  (60, 16000)
Test:  (60, 16000)


In [27]:
train_X[0][:5]

array([-11., -21., -25., -42., -33.])

#### MFCCs

We can also do the same for the 1D mel frequency cepstrum (with a different vector length - n).

In [30]:
n = len(utils.extract_mfccs(path_to_wav))
n

40

In [31]:
train_X_mfccs = utils.get_X_mfccs(sample_train_wavs, n)
cv_X_mfccs = utils.get_X_mfccs(sample_cv_wavs, n)
test_X_mfccs = utils.get_X_mfccs(sample_test_wavs, n)

print("Train mfccs: ", train_X_mfccs.shape)
print("CV mfccs: ", cv_X_mfccs.shape)
print("Test mfccs: ",test_X_mfccs.shape)

Train mfccs:  (240, 40)
CV mfccs:  (60, 40)
Test mfccs:  (60, 40)


In [32]:
train_X_mfccs[0][:5]

array([-445.35308297,   31.80853315,   -3.24210645,   18.580648  ,
          0.94357346])

#### Mel spectrogam

# CONTINUE HERE

In [37]:
for tmp in sample_train_wavs:
    sr, raw = utils.get_wav_info(tmp)
    tmp2 = librosa.feature.melspectrogram(raw, sr)
    print(tmp2.shape)

(128, 32)
(128, 32)
(128, 32)
(128, 32)
(128, 32)
(128, 22)
(128, 32)
(128, 32)
(128, 32)
(128, 30)
(128, 32)
(128, 32)
(128, 32)
(128, 32)
(128, 32)
(128, 32)
(128, 32)
(128, 32)
(128, 32)
(128, 27)
(128, 32)
(128, 32)
(128, 32)
(128, 32)
(128, 32)
(128, 32)
(128, 32)
(128, 32)
(128, 32)
(128, 32)
(128, 25)
(128, 32)
(128, 30)
(128, 32)
(128, 32)
(128, 32)
(128, 32)
(128, 32)
(128, 32)
(128, 32)
(128, 32)
(128, 25)
(128, 32)
(128, 32)
(128, 32)
(128, 32)
(128, 32)
(128, 32)
(128, 32)
(128, 32)
(128, 32)
(128, 32)
(128, 32)
(128, 32)
(128, 32)
(128, 32)
(128, 32)
(128, 32)
(128, 32)
(128, 32)
(128, 32)
(128, 32)
(128, 32)
(128, 30)
(128, 32)
(128, 32)
(128, 32)
(128, 32)
(128, 32)
(128, 32)
(128, 32)
(128, 32)
(128, 32)
(128, 27)
(128, 32)
(128, 32)
(128, 27)
(128, 32)
(128, 32)
(128, 32)
(128, 32)
(128, 32)
(128, 32)
(128, 32)
(128, 32)
(128, 32)
(128, 32)
(128, 32)
(128, 32)
(128, 32)
(128, 32)
(128, 31)
(128, 32)
(128, 32)
(128, 32)
(128, 32)
(128, 30)
(128, 32)
(128, 32)
(128, 32)


In [79]:
def get_X_mel_spectrogram(list_of_paths, shape=(128, 32)):

    # get shape data
    rows = len(list_of_paths)

    # create placeholder
    result = np.array([])

    # go through every file path in the list
    for path_to_wav in list_of_paths:
        
        # get raw array of signed ints
        sr, raw_data = utils.get_wav_info(path_to_wav)
        mel_spectrogram = librosa.feature.melspectrogram(raw_data, sr)

        # some of our sample have less (or slightly more) than the expected amount of values,
        # so let's adjust them
        placeholder = np.array([])
        for row in mel_spectrogram:
            
            # trim to fixed length
            row = row[:shape[1]]

            # pad with zeros, calculating amount of padding needed
            padding = shape[1] - len(row)
            row = np.pad(row, (0, padding), mode='constant', constant_values=0)

            # append the new row
            placeholder = np.append(placeholder, row)
        
        # append the new unrolled matrix to the final result array
        result = np.append(result, placeholder)
    
    # reshape into a 3-dim matrix
    result = np.reshape(result, (len(list_of_paths), shape[0], shape[1]))
    
    return result

In [80]:
tmp = get_X_mel_spectrogram(sample_train_wavs, shape=(128, 32))
tmp.shape

(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(4096,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(3

(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(4096,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(3

(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(4096,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(3

(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(4096,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(3

(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(4096,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(3

...

(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(4096,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(4096,)
(32,)
(32,)


(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(4096,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(3

(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(4096,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(3

(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(4096,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(3

(240, 128, 32)

In [ ]:
train_X_mel_spectrogram = utils.get_X_mel_spectrogram(sample_train_wavs)
cv_X_mel_spectrogram = utils.get_X_mel_spectrogram(sample_cv_wavs)
test_X_mel_spectrogram = utils.get_X_mel_spectrogram(sample_test_wavs)

print("Train mel spectrogram: ", train_X_mel_spectrogram.shape)
print("CV mel spectrogram: ", cv_X_mel_spectrogram.shape)
print("Test mel spectrogram: ",test_X_mel_spectrogram.shape)

In [ ]:
tratrain_X_mel_spectrogram[0][:5]

#### FFT (Fast Fourier Transform)

#### Tempogram

## Train simple models
We will start by training the simplest models and then try out more and more complex architectures, aiming for the highest possible accuracy and F1 score.

The simplest model we can try is a linear model, which we can obtain by using the Keras Dense layer followed by an activation function such as softmax (as in our case categories are mutually exclusive).

#### Linear Model
We'll need to keep track of the dimensions that we pass into our models, so lets assign their values to separate variables.

In [21]:
# we'll need the number of parameters and the output categories
num_features = train_X.shape[1]
num_categories = train_y.shape[1]
print("Input features: {}\nCategories to predict: {}".format(num_features, num_categories))

Input features: 16000
Categories to predict: 12


In [44]:
# design & compile the model
linear_model = Sequential([
    Dense(input_shape=(num_features,), units = num_categories, activation="softmax")
])

# we choose the Adam optimizer with a specific learning rate
linear_model.compile(Adam(lr=0.001),loss="categorical_crossentropy", metrics=["accuracy"])

In [45]:
# let's evaluate our loss before fitting the model
initial_score = linear_model.evaluate(test_X, test_y, verbose=0)
categorical_crossentropy = initial_score[0]
accuracy = initial_score[1]

print("Based on random weights initialization (values will change everytime you compile the model)\nCategorical crossentropy (loss): {:.4f}\nAccuracy: {:.2f}".format(categorical_crossentropy, accuracy))

Based on random weights initialization (values will change everytime you compile the model)
Categorical crossentropy (loss): 14.7749
Accuracy: 0.08


Let's fit our simple linear model for a couple of epochs and see the **F1 score** and **accuracy**.

In [46]:
# we pass our training data and our cross-validation data to see if we're not overfitting
history = linear_model.fit(train_X, train_y, batch_size=32, epochs=5, validation_data=(cv_X, cv_y))

Train on 240 samples, validate on 60 samples
Epoch 1/5
240/240 [==============================] - 0s - loss: 14.5064 - acc: 0.1000 - val_loss: 14.7749 - val_acc: 0.0833
Epoch 2/5
240/240 [==============================] - 0s - loss: 14.5734 - acc: 0.0958 - val_loss: 15.0436 - val_acc: 0.0667
Epoch 3/5
240/240 [==============================] - 0s - loss: 14.5063 - acc: 0.1000 - val_loss: 15.0436 - val_acc: 0.0667
Epoch 4/5
240/240 [==============================] - 0s - loss: 14.4391 - acc: 0.1042 - val_loss: 15.0436 - val_acc: 0.0667
Epoch 5/5
240/240 [==============================] - 0s - loss: 14.5734 - acc: 0.0958 - val_loss: 15.0436 - val_acc: 0.0667


In [47]:
# show latest results
best_training_accuracy = max(history.history["acc"])
best_validation_accuracy = max(history.history["val_acc"])
print("Best scores\nTrain acc: {:.4f}\nCV acc: {:.4f}".format(best_training_accuracy, best_validation_accuracy))

Best scores
Train acc: 0.1042
CV acc: 0.0833


Depending on the random initialization of weights we should have an **accuracy** score within 0.05 and 0.15 on both the training and cross-validation set. Let's also calculate the **F1 score**.

In [48]:
# first use the model to predict the labels
pred_cv_y = linear_model.predict(cv_X, batch_size=32)

In [49]:
pred_cv_y.shape

(60, 12)

In [50]:
# check if shape matches expectation (number of examples, number of categories to predict)
pred_cv_y.shape

(60, 12)

In [51]:
# we use softmax to get a result towards one-hot encoding, but not all rows will be just zeroes and one 1
pred_cv_y[:10]

array([[0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

So before we pass our predictions to the sklearn's f1 score function we need to make sure that all of our rows are actually one-hot encoded.

In [52]:
pred_cv_y = utils.one_hot_encode(pred_cv_y)
pred_cv_y[:10]

array([[0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [53]:
# we can also use sklearn directly to get accuracy
sk_cv_accuracy = accuracy_score(cv_y, pred_cv_y)
print("Final linear model CV accuracy via sklearn: {:.4f}".format(sk_cv_accuracy))

Final linear model CV accuracy via sklearn: 0.0667


In [54]:
# because we're dealing with a mutliclass classification challenge, we need to change the default value of average
# (which is binary)
cv_f1_score = f1_score(cv_y, pred_cv_y, average="weighted")
print("Linear model f1 score (CV): {:.4f}".format(cv_f1_score))

Linear model f1 score (CV): 0.0676


In summary, our accuracy and F1 score for the simplest possible model fall within 0.05 - 0.15. This is our earliest benchmark to beat, and it's not much better than **random guessing**, which given 12 categories would give us an accuracy of 0.08333.

#### Random Forest
It is also useful to try other ML methods before jumping into neural networks and deep learning. Random Forests are a simple but very often quite effective (and computationally inexpensive) method of obtaining a good benchmark.

For the sklearn implementation of Random Forest we actually do not want our target to be one-hot encoded.

In [62]:
# reverse the one-hot encoding
rf_train_y = utils.reverse_one_hot_encoding(train_y)
rf_cv_y = utils.reverse_one_hot_encoding(cv_y)
rf_test_y = utils.reverse_one_hot_encoding(test_y)

In [66]:
rand_forest = RandomForestClassifier(max_depth=7, random_state=0)
rand_forest.fit(train_X, rf_train_y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=7, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [67]:
rf_predicted_cv_y = rand_forest.predict(cv_X)
rf_predicted_cv_y

array([10.,  8.,  9.,  2.,  5., 12., 12.,  9.,  4.,  5.,  4.,  9.,  6.,
        2., 11.,  8., 10.,  8.,  7.,  9., 11., 12.,  7.,  4.,  9.,  2.,
        3.,  1.,  5.,  6.,  8., 12.,  3.,  6., 10.,  2.,  3.,  9.,  1.,
        5., 12.,  9.,  3.,  5.,  7.,  6.,  5.,  3.,  3.,  6., 11.,  4.,
       11.,  9.,  9.,  2.,  3.,  2.,  7., 10.])

In [68]:
# calculate accuracy and F1 for Random Forest
rf_cv_f1_score = f1_score(rf_cv_y, rf_predicted_cv_y, average="weighted")
rf_cv_accuracy = accuracy_score(rf_cv_y, rf_predicted_cv_y)

print("Random forest f1 score (CV): {:.3f}".format(rf_cv_f1_score))
print("Random forest accuracy (CV): {:.3f}".format(rf_cv_accuracy))

Random forest f1 score (CV): 0.066
Random forest accuracy (CV): 0.067


For the Random Forest method, using only default parameters (except for a max depth of 7), we are getting an **F1 score and accuracy around 0.11**. Slightly better than random, nowhere near good enough.

#### MFCCS Linear Model & Random Forest
Let's see if our methods result in a higher score for the 1D mel frequency cepstrum coefficients.

In [69]:
# design & compile the model
mfcc_linear_model = Sequential([
    Dense(input_shape=(num_features,), units = num_categories, activation="softmax")
])

# we choose the Adam optimizer with a specific learning rate
mfcc_linear_model.compile(Adam(lr=0.001),loss="categorical_crossentropy", metrics=["accuracy"])

In [70]:
# let's evaluate our loss before fitting the model
initial_score = mfcc_linear_model.evaluate(cv_X_mfccs, cv_y, verbose=0)
categorical_crossentropy = initial_score[0]
accuracy = initial_score[1]

print("MFCCs\nBased on random weights initialization (values will change everytime you compile the model)\nCategorical crossentropy (loss): {:.4f}\nAccuracy: {:.2f}".format(categorical_crossentropy, accuracy))

MFCCs
Based on random weights initialization (values will change everytime you compile the model)
Categorical crossentropy (loss): 6.0558
Accuracy: 0.10


In [71]:
# we pass our training data and our cross-validation data to see if we're not overfitting
history = mfcc_linear_model.fit(train_X_mfccs, train_y, batch_size=32, epochs=10, validation_data=(cv_X_mfccs, cv_y))

Train on 240 samples, validate on 60 samples
Epoch 1/10
240/240 [==============================] - 0s - loss: 4.1227 - acc: 0.0917 - val_loss: 2.9069 - val_acc: 0.0667
Epoch 2/10
240/240 [==============================] - 0s - loss: 2.5801 - acc: 0.1583 - val_loss: 2.6871 - val_acc: 0.2167
Epoch 3/10
240/240 [==============================] - 0s - loss: 2.4474 - acc: 0.2000 - val_loss: 2.5900 - val_acc: 0.1500
Epoch 4/10
240/240 [==============================] - 0s - loss: 2.1807 - acc: 0.3458 - val_loss: 2.4405 - val_acc: 0.1667
Epoch 5/10
240/240 [==============================] - 0s - loss: 2.0224 - acc: 0.3625 - val_loss: 2.3072 - val_acc: 0.2333
Epoch 6/10
240/240 [==============================] - 0s - loss: 1.9372 - acc: 0.3875 - val_loss: 2.3053 - val_acc: 0.1667
Epoch 7/10
240/240 [==============================] - 0s - loss: 1.8468 - acc: 0.4250 - val_loss: 2.2772 - val_acc: 0.2167
Epoch 8/10
240/240 [==============================] - 0s - loss: 1.7642 - acc: 0.4500 - val_lo

We can quickly observe that the linear model based on the mfcc data is much better at fitting the training data - getting to a train accuracy of 0.44 after 10 epochs and a validation accuracy of around 0.233 (compared to the raw data linear model not progressing beyond train and cv accuracy of 0.15).

Let's run the linear model for 10 more epochs to see if we can get a better cv accuracy, despite clearly already overfitting.

In [72]:
# 10 more epochs
history = mfcc_linear_model.fit(train_X_mfccs, train_y, batch_size=32, epochs=10, validation_data=(cv_X_mfccs, cv_y))

Train on 240 samples, validate on 60 samples
Epoch 1/10
240/240 [==============================] - 0s - loss: 1.6312 - acc: 0.5250 - val_loss: 2.2314 - val_acc: 0.2500
Epoch 2/10
240/240 [==============================] - 0s - loss: 1.5793 - acc: 0.5250 - val_loss: 2.2600 - val_acc: 0.3000
Epoch 3/10
240/240 [==============================] - 0s - loss: 1.5587 - acc: 0.5583 - val_loss: 2.2378 - val_acc: 0.3000
Epoch 4/10
240/240 [==============================] - 0s - loss: 1.5395 - acc: 0.5375 - val_loss: 2.2464 - val_acc: 0.3000
Epoch 5/10
240/240 [==============================] - 0s - loss: 1.4858 - acc: 0.5625 - val_loss: 2.2713 - val_acc: 0.3167
Epoch 6/10
240/240 [==============================] - 0s - loss: 1.4735 - acc: 0.5833 - val_loss: 2.2500 - val_acc: 0.3167
Epoch 7/10
240/240 [==============================] - 0s - loss: 1.4589 - acc: 0.5667 - val_loss: 2.2652 - val_acc: 0.3333
Epoch 8/10
240/240 [==============================] - 0s - loss: 1.4308 - acc: 0.5958 - val_lo

Indeed we managed to reach a cv accuracy of over 0.3. If you do further experiments you can also stop on an epoch where the cv accuracy was around 0.35.

Let's see what accuracy and F1 score we can obtain from this model on the cv set.

In [73]:
# first use the model to predict the labels
mfccs_pred_cv_y = mfcc_linear_model.predict(cv_X_mfccs, batch_size=32)
mfccs_pred_cv_y.shape

(60, 12)

In [74]:
# make sure it's one-hot encoded
mfccs_pred_cv_y = utils.one_hot_encode(mfccs_pred_cv_y)
mfccs_pred_cv_y.shape

(60, 12)

In [75]:
# we can also use sklearn directly to get accuracy
mfccs_cv_accuracy = accuracy_score(cv_y, mfccs_pred_cv_y)
mfccs_cv_f1_score = f1_score(cv_y, mfccs_pred_cv_y, average="weighted")
print("MFCCs Linear model accuracy via sklearn (CV): {:.4f}".format(mfccs_cv_accuracy))
print("MFCCs Linear model f1 score (CV): {:.4f}".format(mfccs_cv_f1_score))

MFCCs Linear model accuracy via sklearn (CV): 0.3333
MFCCs Linear model f1 score (CV): 0.3189


We have a new benchmark - the **linear model based on the mfccs** has an accuracy and F1 score of around **0.3**.

Let's try the random forest approach on the mfccs.

In [76]:
# initialize the random forest and fit it to mfcc X
mfccs_rand_forest = RandomForestClassifier(max_depth=7, random_state=0)
mfccs_rand_forest.fit(train_X_mfccs, rf_train_y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=7, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [77]:
mfccs_rf_predicted_cv_y = mfccs_rand_forest.predict(cv_X_mfccs)
mfccs_rf_predicted_cv_y

array([ 4.,  3., 11.,  6., 10., 10.,  5.,  8., 12., 11.,  9.,  9.,  7.,
        5.,  6.,  3., 12.,  7.,  3.,  6.,  6., 12.,  8.,  6., 12.,  2.,
        8.,  3.,  7.,  5.,  5.,  3.,  5.,  3.,  7.,  3.,  2.,  6.,  8.,
        4., 12.,  9., 12.,  9., 11.,  7.,  1.,  9., 12.,  3., 11., 11.,
       11., 11.,  2., 10.,  3.,  6., 11., 12.])

In [78]:
# calculate accuracy and F1 for Random Forest
mfccs_rf_cv_f1_score = f1_score(rf_cv_y, mfccs_rf_predicted_cv_y, average="weighted")
mfccs_rf_cv_accuracy = accuracy_score(rf_cv_y, mfccs_rf_predicted_cv_y)

print("Random forest f1 score (CV): {:.3f}".format(mfccs_rf_cv_f1_score))
print("Random forest accuracy (CV): {:.3f}".format(mfccs_rf_cv_accuracy))

Random forest f1 score (CV): 0.133
Random forest accuracy (CV): 0.150


The Random Forest model with all default parameters and a max_depth of 7 is able to make more accurate predictions based on the mfccs data than on the raw data, but not more accuracte than the linear model.

Let's experiment a little with some of the other parameters of our Random Forest to see if we can get a better result than our mfccs linear model.

In [79]:
# initialize the random forest and fit it to mfcc X
optimized_mfccs_rand_forest = RandomForestClassifier(
    n_estimators=50,
    max_depth=7,
    max_features = 3000,
    random_state=0)
optimized_mfccs_rand_forest.fit(train_X_mfccs, rf_train_y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=7, max_features=3000, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [80]:
optimized_mfccs_rf_predicted_cv_y = optimized_mfccs_rand_forest.predict(cv_X_mfccs)
optimized_mfccs_rf_predicted_cv_y

array([ 1.,  6.,  9.,  1.,  4.,  3.,  6.,  7., 12., 11.,  1.,  8.,  9.,
        9.,  4.,  8.,  8.,  7.,  2.,  6., 12.,  9., 12.,  5., 12.,  4.,
       12.,  2.,  8.,  2.,  8.,  8.,  3.,  7.,  3.,  8.,  2.,  7.,  3.,
       10., 12., 12., 12., 10.,  3.,  2.,  3.,  4., 12.,  7., 11., 11.,
       11., 11., 11.,  2.,  3.,  6., 12.,  1.])

In [81]:
# calculate accuracy and F1 for Random Forest
optimized_mfccs_rf_cv_f1_score = f1_score(rf_cv_y, optimized_mfccs_rf_predicted_cv_y, average="weighted")
optimized_mfccs_rf_cv_accuracy = accuracy_score(rf_cv_y, optimized_mfccs_rf_predicted_cv_y)

print("Random forest f1 score (CV): {:.3f}".format(optimized_mfccs_rf_cv_f1_score))
print("Random forest accuracy (CV): {:.3f}".format(optimized_mfccs_rf_cv_accuracy))

Random forest f1 score (CV): 0.182
Random forest accuracy (CV): 0.183


After a little bit of tweaking we can get a Random Forest with accuracy and F1 score approaching 0.3, right around our current benchmark.

In [82]:
# set benchmark
best_cv_acc = 0.3

## Train Neural Networks
Now that we have a benchmark obtained via simple linear and Random Forest models we can proceed towards trying to outdo it with MLPs and deep learning models.

#### MLP - multi-layer perceptron
Let's start with the simplest possible neural network of just 2 dense layers. We'll be working only on the mfccs data from now on, as it tends to produce better results. We will also add **batch normalization** and **dropout** to reduce overfitting.

In [83]:
# design & compile the model
num_nodes = 1000
mlp = Sequential([
    Dense(input_shape=(num_features,), units = num_nodes, activation="relu"),
    BatchNormalization(),
    Dropout(0.8),
    Dense(num_categories, activation='softmax')
])

# we choose the Adam optimizer with a specific learning rate
mlp.compile(Adam(lr=0.001),loss="categorical_crossentropy", metrics=["accuracy"])

In [84]:
# let's train 
for i in range(30):
    print("Actual epoch: {}".format(i + 1))
    mlp_results = mlp.fit(train_X_mfccs, train_y, batch_size=32, epochs=1, validation_data=(cv_X_mfccs, cv_y))
    # stop if we exceed previous best (benchmark)
    current_cv_acc = mlp_results.history["val_acc"][0] 
    if current_cv_acc > best_cv_acc:
        break

Actual epoch: 1
Train on 240 samples, validate on 60 samples
Epoch 1/1
240/240 [==============================] - 1s - loss: 3.8312 - acc: 0.1000 - val_loss: 3.7503 - val_acc: 0.1167
Actual epoch: 2
Train on 240 samples, validate on 60 samples
Epoch 1/1
240/240 [==============================] - 1s - loss: 3.2255 - acc: 0.2125 - val_loss: 4.2612 - val_acc: 0.1667
Actual epoch: 3
Train on 240 samples, validate on 60 samples
Epoch 1/1
240/240 [==============================] - 1s - loss: 2.7562 - acc: 0.2917 - val_loss: 3.9832 - val_acc: 0.2333
Actual epoch: 4
Train on 240 samples, validate on 60 samples
Epoch 1/1
240/240 [==============================] - 1s - loss: 2.5700 - acc: 0.2750 - val_loss: 3.7155 - val_acc: 0.2333
Actual epoch: 5
Train on 240 samples, validate on 60 samples
Epoch 1/1
240/240 [==============================] - 1s - loss: 2.5591 - acc: 0.2875 - val_loss: 3.9003 - val_acc: 0.1833
Actual epoch: 6
Train on 240 samples, validate on 60 samples
Epoch 1/1
240/240 [=====

In [85]:
# show latest results
last_training_accuracy = max(mlp_results.history["acc"])
last_validation_accuracy = max(mlp_results.history["val_acc"])
print("Last MLP scores\nTrain acc: {:.4f}\nCV acc: {:.4f}".format(last_training_accuracy, last_validation_accuracy))

Last MLP scores
Train acc: 0.3833
CV acc: 0.3000


In [86]:
# predict and one-hot encode
mlp_pred_cv_y = mlp.predict(cv_X_mfccs, batch_size=32)
mlp_pred_cv_y = utils.one_hot_encode(mlp_pred_cv_y)
mlp_pred_cv_y.shape

(60, 12)

In [87]:
# we can also use sklearn directly to get accuracy
mlp_cv_accuracy = accuracy_score(cv_y, mlp_pred_cv_y)
mlp_cv_f1_score = f1_score(cv_y, mlp_pred_cv_y, average="weighted")
print("MLP accuracy via sklearn (CV): {:.4f}".format(mlp_cv_accuracy))
print("MLP f1 score (CV): {:.4f}".format(mlp_cv_f1_score))

MLP accuracy via sklearn (CV): 0.3000
MLP f1 score (CV): 0.2476


We can see that a simple MLP model reaches a very similar accuracy score to our previous benchmark of 0.3. Both this one and the previous ones can be tuned to reach approximately 0.35 but let's save fine-tuning for when we have a more promising approach - we are also already overfitting.

#### Deep Neural Networks
Let's try adding more layers to capture more complex interactions.

In [88]:
dnn = Sequential([
    Dense(input_shape=(num_features,), units = 1500, activation="relu"),
    BatchNormalization(),
    Dropout(0.87),
    Dense(1000, activation="relu"),
    BatchNormalization(),
    Dropout(0.87),
    Dense(num_categories, activation='softmax')
])

# we choose the Adam optimizer with a specific learning rate
dnn.compile(Adam(lr=0.001),loss="categorical_crossentropy", metrics=["accuracy"])

In [89]:
# let's train 
for i in range(30):
    print("Actual epoch: {}".format(i + 1))
    dnn_results = dnn.fit(train_X_mfccs, train_y, batch_size=32, epochs=1, validation_data=(cv_X_mfccs, cv_y))
    # stop if we exceed or meet previous best (benchmark) - you can then re-run to see if we're overfitting or not
    current_cv_acc = dnn_results.history["val_acc"][0] 
    if current_cv_acc >= best_cv_acc:
        break

Actual epoch: 1
Train on 240 samples, validate on 60 samples
Epoch 1/1
240/240 [==============================] - 2s - loss: 5.8975 - acc: 0.0583 - val_loss: 4.5369 - val_acc: 0.0500
Actual epoch: 2
Train on 240 samples, validate on 60 samples
Epoch 1/1
240/240 [==============================] - 2s - loss: 5.2998 - acc: 0.1458 - val_loss: 5.8575 - val_acc: 0.1167
Actual epoch: 3
Train on 240 samples, validate on 60 samples
Epoch 1/1
240/240 [==============================] - 2s - loss: 5.0662 - acc: 0.1333 - val_loss: 6.6372 - val_acc: 0.1167
Actual epoch: 4
Train on 240 samples, validate on 60 samples
Epoch 1/1
240/240 [==============================] - 2s - loss: 5.0278 - acc: 0.1458 - val_loss: 6.7043 - val_acc: 0.1500
Actual epoch: 5
Train on 240 samples, validate on 60 samples
Epoch 1/1
240/240 [==============================] - 2s - loss: 4.9471 - acc: 0.1417 - val_loss: 6.3128 - val_acc: 0.1333
Actual epoch: 6
Train on 240 samples, validate on 60 samples
Epoch 1/1
240/240 [=====

In [92]:
# show latest results
last_training_accuracy = max(dnn_results.history["acc"])
last_validation_accuracy = max(dnn_results.history["val_acc"])
print("Last DNN scores\nTrain acc: {:.4f}\nCV acc: {:.4f}".format(last_training_accuracy, last_validation_accuracy))

Last DNN scores
Train acc: 0.2542
CV acc: 0.1833


In [93]:
# predict and one-hot encode
dnn_pred_cv_y = dnn.predict(cv_X_mfccs, batch_size=32)
dnn_pred_cv_y = utils.one_hot_encode(dnn_pred_cv_y)
dnn_pred_cv_y.shape

(60, 12)

In [94]:
# we can also use sklearn directly to get accuracy
dnn_cv_accuracy = accuracy_score(cv_y, dnn_pred_cv_y)
dnn_cv_f1_score = f1_score(cv_y, dnn_pred_cv_y, average="weighted")
print("DNN accuracy via sklearn (CV): {:.4f}".format(dnn_cv_accuracy))
print("DNN f1 score (CV): {:.4f}".format(dnn_cv_f1_score))

DNN accuracy via sklearn (CV): 0.1833
DNN f1 score (CV): 0.1868


#### Convolutional Models
Seems we're stuck around 0.3 accuracy. That makes sense because the actual "no" and other words may come at any place in the vector, we can't really keep being attached to specific indexes when training (which we currently are). Let's try convolutional layers, which can find certain patterns regardless of whether they appear at the start or end of the file.

In [95]:
# In order to use convolutions we have reshape our X -> expand it to 3 dimensions
conv_train_X_mfccs = np.expand_dims(train_X_mfccs, axis=2)
conv_train_X_mfccs.shape

(240, 16000, 1)

In [96]:
# repeat for cv & test
conv_cv_X_mfccs = np.expand_dims(cv_X_mfccs, axis=2)
conv_test_X_mfccs = np.expand_dims(test_X_mfccs, axis=2)

In [97]:
cnn1 = Sequential([
        Convolution1D(input_shape=(num_features, 1), kernel_size=32, filters=8, padding="same", activation="relu"),
        Dropout(0.1),
        MaxPooling1D(),
        Convolution1D(kernel_size=64, filters=16, padding="same", activation="relu"),
        Dropout(0.1),
        MaxPooling1D(),
        Flatten(),
        Dense(500, activation="relu"),
        Dropout(.6),
        Dense(num_categories, activation="softmax")
    ])

cnn1.compile(Adam(lr=0.001),loss="categorical_crossentropy", metrics=["accuracy"])

This CNN architecture should get to 0.367 accuracy around the 35 epoch and then start to overfit.

In [98]:
cnn1_results = cnn1.fit(conv_train_X_mfccs, train_y, batch_size=32, epochs=60, validation_data=(conv_cv_X_mfccs, cv_y))

Train on 240 samples, validate on 60 samples
Epoch 1/60
240/240 [==============================] - 8s - loss: 4.8245 - acc: 0.0750 - val_loss: 2.5706 - val_acc: 0.1167
Epoch 2/60
240/240 [==============================] - 8s - loss: 2.5976 - acc: 0.1125 - val_loss: 2.4629 - val_acc: 0.1167
Epoch 3/60
240/240 [==============================] - 7s - loss: 2.4707 - acc: 0.1208 - val_loss: 2.4688 - val_acc: 0.0833
Epoch 4/60
128/240 [===============>..............] - ETA: 3s - loss: 2.4641 - acc: 0.1484

KeyboardInterrupt: 

In [99]:
# show best results
best_training_accuracy = max(cnn1_results.history["acc"])
best_validation_accuracy = max(cnn1_results.history["val_acc"])
print("Best CNN 1 scores\nTrain acc: {:.4f}\nCV acc: {:.4f}".format(best_training_accuracy, best_validation_accuracy))

NameError: name 'cnn1_results' is not defined

In [100]:
# predict and one-hot encode
cnn1_pred_cv_y = cnn1.predict(conv_cv_X_mfccs, batch_size=32)
cnn1_pred_cv_y = utils.one_hot_encode(cnn1_pred_cv_y)
cnn1_pred_cv_y.shape

(60, 12)

In [101]:
# we can also use sklearn directly to get accuracy
cnn1_cv_accuracy = accuracy_score(cv_y, cnn1_pred_cv_y)
cnn1_cv_f1_score = f1_score(cv_y, cnn1_pred_cv_y, average="weighted")
print("CNN 1 accuracy via sklearn (CV): {:.4f}".format(cnn1_cv_accuracy))
print("CNN 1 f1 score (CV): {:.4f}".format(cnn1_cv_f1_score))

CNN 1 accuracy via sklearn (CV): 0.1500
CNN 1 f1 score (CV): 0.0981


Let's increase the kernel size - patterns in speech mighr require more than e.g. 32 single samplings to be recognizable.

In [102]:
cnn2 = Sequential([
        Convolution1D(input_shape=(num_features, 1), kernel_size=256, filters=32, padding="same", activation="relu"),
        Dropout(0.2),
        MaxPooling1D(),
        Convolution1D(kernel_size=512, filters=32, padding="same", activation="relu"),
        Dropout(0.2),
        MaxPooling1D(),
        Flatten(),
        Dense(500, activation="relu"),
        Dropout(.6),
        Dense(num_categories, activation="softmax")
    ])

cnn2.compile(Adam(lr=0.001),loss="categorical_crossentropy", metrics=["accuracy"])

In [103]:
cnn2_results = cnn2.fit(conv_train_X_mfccs, train_y, batch_size=32, epochs=50, validation_data=(conv_cv_X_mfccs, cv_y))

Train on 240 samples, validate on 60 samples
Epoch 1/50
240/240 [==============================] - 169s - loss: 6.7903 - acc: 0.0458 - val_loss: 2.4849 - val_acc: 0.0833
Epoch 2/50
 64/240 [=======>......................] - ETA: 115s - loss: 2.4854 - acc: 0.0469

KeyboardInterrupt: 

#### Recurrent Models
We can also try to take advantage of the architectures specifically designed for time sequences: RNNs. We will start with the basic keras implementations of simple RNN and then move on to GRUs & LSTMs.

In [ ]:
rnn_1 = Sequential([
        SimpleRNN(input_shape=(num_features, 1), units=100, activation='relu'),
        Dense(500, activation="relu"),
        BatchNormalization(),
        Dropout(.7),
        Dense(num_categories, activation="softmax")
    ])

rnn_1.compile(Adam(lr=0.001),loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
rnn_1_results = rnn_1.fit(conv_train_X_mfccs, train_y, batch_size=32, epochs=50, validation_data=(conv_cv_X_mfccs, cv_y))

## Action plan
X) turn the sample data into numpy arrays with X and y normally <br>
X) turn sample data into numpy arrays with X and y via mfccs<br>
X) Use linear model? (towards first benchmark)<br>
X) Use random forest?<br>
X) Use MLP<br>
X) Use multiple dense layers<br>
4c) Use convolutions (try the increased kernel sie that takes 400s per epoch)<br>
4d) USE RNN -> like in Nietzsche [https://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/]<br>
5) Add preprocessing and test a couple of the best models<br>

6) Consider splitting the work on images into separate notebook depending on how bulky this gets<br>
7) Experiments on images without data augmentation<br>
8) Experiments on images with data augmentation<br>

9) Decide on e.g. 3 most promising methods<br>

And then:<br>
10) Move to writing the most promising models in tensorflow<br>
11) Include tensorboard visualization of training & graph<br>
12) Code for turning results into kaggle format of results to get score<br>
13) Obtain a good score on kaggle<br>
14) Re-read everything from start to finish and adjust<br>
15) Write a good Readme for markdown<br>
16) Add to CV<br>